<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/ppn_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPN Parameter Extraction Experiment

This experiment extracts Post-Newtonian Parameter (PPN) γ and β from our computational-capacity metric and compares against General Relativity predictions.

## Setup: Clone Repository and Install Dependencies

In [ ]:
# Clone and setup (idempotent with forced refresh)
import os, sys, subprocess, shutil, pathlib
REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

# Force fresh clone to avoid caching issues
if pathlib.Path(REPO_NAME).exists():
    !rm -rf $REPO_NAME
    
!git clone $REPO_URL

%cd $REPO_NAME

# Install package
if (pathlib.Path("pyproject.toml")).exists():
    !pip install -e .

# Set random seed for reproducibility
import numpy as np, random
np.random.seed(42)
random.seed(42)

## Run PPN Parameter Extraction

In [ ]:
from experiments.run_ppn import main
main()

## Display Results

In [ ]:
from IPython.display import Image, display
from pathlib import Path

# Display generated plots
output_dir = Path("experiments/out/ppn")

print("PPN Parameters vs Constitutive Parameter κ:")
display(Image(str(output_dir / "ppn_parameters.png")))

print("\nN to Newtonian Potential Mapping:")
display(Image(str(output_dir / "n_to_phi_mapping.png")))

print("\nSpecial Relativity Limit Test:")
display(Image(str(output_dir / "sr_limit.png")))

print("\nConstitutive Law B(N) for Different κ:")
display(Image(str(output_dir / "constitutive_law.png")))

# Display numerical results
print("\nNumerical Results:")
with open(output_dir / "results.txt", 'r') as f:
    print(f.read())

## Optional: Interactive Parameter Exploration

In [ ]:
# Interactive exploration of the constitutive law
import numpy as np
import matplotlib.pyplot as plt
from tacc import ppn, constitutive

# Widget for interactive exploration (works in Colab)
def explore_kappa(kappa_value=2.0):
    """Explore different kappa values"""
    
    # Extract PPN parameters
    gamma, beta = ppn.extract_ppn_params(kappa_value)
    
    print(f"κ = {kappa_value:.2f}")
    print(f"γ = {gamma:.4f} (GR: 1.0000)")
    print(f"β = {beta:.4f} (GR: 1.0000)")
    print(f"Deviation from GR: Δγ = {abs(gamma-1.0):.4f}, Δβ = {abs(beta-1.0):.4f}")
    
    # Plot constitutive law
    N_range = np.linspace(0.5, 2.0, 200)
    B_values = constitutive.B_of_N(N_range, kappa_value)
    
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.plot(N_range, B_values, 'b-', linewidth=2, label=f'κ={kappa_value}')
    plt.axvline(x=1.0, color='k', linestyle=':', alpha=0.7, label='N=1 (GR limit)')
    plt.axhline(y=1.0, color='k', linestyle=':', alpha=0.7)
    plt.xlabel('N (Computational Capacity)')
    plt.ylabel('B(N) = exp[-κ(1-N)]')
    plt.title(f'Constitutive Law (κ={kappa_value})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot PPN parameter relationship
    kappa_range = np.linspace(0.5, 4.0, 100)
    gamma_range = kappa_range / 2  # γ = κ/2
    
    plt.subplot(1, 2, 2)
    plt.plot(kappa_range, gamma_range, 'r-', linewidth=2, label='γ = κ/2')
    plt.axhline(y=1.0, color='k', linestyle='--', alpha=0.7, label='GR (γ=1)')
    plt.axvline(x=kappa_value, color='b', linestyle=':', alpha=0.7, label=f'Current κ={kappa_value}')
    plt.scatter([kappa_value], [gamma], color='red', s=100, zorder=10)
    plt.xlabel('κ (Constitutive Parameter)')
    plt.ylabel('γ (PPN Parameter)')
    plt.title('PPN γ vs κ Relationship')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Try different values
print("Exploring different κ values:")
for kappa in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
    explore_kappa(kappa)
    print("\n" + "="*50 + "\n")

## Troubleshooting

**Common Issues:**
- If plots don't display, ensure matplotlib is installed
- Import errors indicate repository cloning issues
- **TypeError: only length-1 arrays can be converted to Python scalars**: This error occurs when Colab caches an old version of the repository. The setup cell above now forces a fresh clone to avoid this issue. If you still see this error, restart the runtime (Runtime → Restart runtime) and run all cells again.

**Key Insights:**
- In our model, β=1 always (built into the theory)
- γ = κ/2, so κ=2 gives exact GR limit γ=1
- The constitutive parameter κ controls the strength of the computational capacity effect
- Near N=1, the model reduces to standard General Relativity